<a href="https://colab.research.google.com/github/ljw-0108/medical-data-analysis/blob/main/02_toxicity_prediction_model_practice_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepChem 설치

Colab 노트북에 DeepChem 설치

tensorflow 기반 모델을 사용할 것이므로 pip install 명령에 [tensorflow]를 추가하여 필요한 종속성도 설치

In [2]:
!pip install deepchem[tensorflow]

## 기타 모듈 설치
- 레거시 옵티마이저 사용을 위한 tf_keras 설치(deepchem ver 2.4.0일 때만)
- rdkit 설치

In [3]:
#!pip install tf_keras
!pip install rdkit

## deepchem 버전 확인

In [4]:
import deepchem as dc
dc.__version__

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.0'

## numpy 버전 확인

In [5]:
import numpy as np
np.__version__

'2.0.2'

## warning 메시지 생략

In [7]:
import warnings
warnings.filterwarnings(action='ignore')

## 데이터셋 다루기

In [8]:
x1 = np.random.random((4,5))
y1 = np.random.random((4,1))

In [9]:
x1

array([[0.79148053, 0.94685479, 0.95219579, 0.85511537, 0.21294188],
       [0.70272731, 0.99903681, 0.66719246, 0.81348716, 0.32399219],
       [0.83073   , 0.12652682, 0.03119832, 0.02503196, 0.67176196],
       [0.62012016, 0.86664327, 0.06868273, 0.21337677, 0.21511221]])

In [10]:
y1

array([[0.71243937],
       [0.96540685],
       [0.92019747],
       [0.41567769]])

In [11]:
dataset_sample = dc.data.NumpyDataset(x1, y1)

In [12]:
print(dataset_sample.X)
print(dataset_sample.y)

[[0.79148053 0.94685479 0.95219579 0.85511537 0.21294188]
 [0.70272731 0.99903681 0.66719246 0.81348716 0.32399219]
 [0.83073    0.12652682 0.03119832 0.02503196 0.67176196]
 [0.62012016 0.86664327 0.06868273 0.21337677 0.21511221]]
[[0.71243937]
 [0.96540685]
 [0.92019747]
 [0.41567769]]


In [13]:
np.array_equal(x1, dataset_sample.X)

True

In [14]:
np.array_equal(y1, dataset_sample.y)

True

# 독성 분자 예측 모델 만들기
DeepChem을 사용해 분자의 독성을 예측하는 모델을 훈련하는 실습 진행

Tox21 독성 데이터 : Tox21 데이터 챌린지 대회에 사용된 데이터로 약물의 독성 예측과 관련된 표적 실험 데이터

## 데이터 불러오기 및 확인

In [15]:
# 데이터 불러오기 및 데이터셋 피처화 (행렬 및 벡터로 변환)
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATION WARNING: please use MorganGenerator
[02:00:15] DEPRECATI

In [16]:
# 데이터셋 분류(학습, 검증, 테스트)
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [17]:
# target (class) : 잠재적 신약에 대한 독성 반응과 관련되어 있다고 여겨지는 단백질
# 각 표적은 실험을 통해 얻은 수치가 포함됨(분자와 해당 단백질간의 결합력)
tox21_tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

In [18]:
len(tox21_tasks)

12

In [19]:
tox21_datasets

(<DiskDataset X.shape: (np.int64(6258), np.int64(1024)), y.shape: (np.int64(6258), np.int64(12)), w.shape: (np.int64(6258), np.int64(12)), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>,
 <DiskDataset X.shape: (np.int64(782), np.int64(1024)), y.shape: (np.int64(782), np.int64(12)), w.shape: (np.int64(782), np.int64(12)), ids: ['CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
  'CSc1nnc(C(C)(C)C)c(=O)n1N'
  'C=C1/C(=C\\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H](C)/C=C/[C@@H](O)C2CC2)C[C@@H](O)C[C@@H]1O'
  ... 'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
  'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
  'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>,
 <DiskDataset X.shape: (np.int64(783), np.int64(1024)), y.shape: (np.int64(783), np.int64(12)), w.shape: (np.int64(783), np.int64(12)), ids: ['CC1(C)S[C@@H]2[C@H](NC

In [20]:
train_dataset.X.shape
#valid_dataset.X.shape
#test_dataset.X.shape

(6258, 1024)

In [21]:
np.shape(train_dataset.y)
#np.shape(valid_dataset.y)
#np.shape(test_dataset.y)

(6258, 12)

## 데이터 전처리
분자 데이터 대부분 표적과 결합하지 않는 데이터로 90% 넘는 데이터의 Label 0.
항상 0을 예측하는 모델을 만들면 정확도가 90%.
즉 데이터셋의 편향 문제가 발생하여 가중치 행렬 조정이 필요

In [22]:
# 불균형 데이터셋 보완
transformers

## 모델 생성 및 훈련

In [23]:
# 모델 생성 및 훈련
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
model.fit(train_dataset, nb_epoch=10)

0.49006166458129885

## 모델 평가

In [24]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)
print(train_scores)
print(test_scores)

{'mean-roc_auc_score': np.float64(0.9585140679961542)}
{'mean-roc_auc_score': np.float64(0.687286492468056)}
